# Advent of Code 2018

This is a notebook of solutions for the [Advent of Code](http://adventofcode.com) for 2018. Each day will have a link to the problem description. Inputs are stored in the 'data' directory.

## Helper functions

In [40]:
import os
import urllib.request

#adapted from from norvig/pytudes repository
def Input(day, year=2018):
    "Open this day's input file."
    directory = 'data/advent{}/'.format(year)
    filename = directory+'day{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        if not os.path.exists(directory):
            os.makedirs(directory)

        urllib.request.urlretrieve("https://raw.githubusercontent.com/elahmo/advent-of-code/master/data/" + filename, filename)
        return Input(day)

def Inputstr(day, year=2018): 
    "The contents of this day's input file as a str."
    return Input(day, year).read().rstrip('\n')

def Inputlist(day, year=2018):
    lines = [line.rstrip('\n') for line in Input(day, year)]
    return lines

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

## [Day 1](https://adventofcode.com/2018/day/1): Chronal Calibration
Task is to find the resulting frequency after applying all the changes.

In [51]:
offsets = Inputlist(1)
total = sum([int(x) for x in offsets])

In [53]:
total

435

### Part Two
It seems the changes are repetitive, the task is to find the frequency that first appears for the second time.

In [96]:
frequency_list = {0} #store the reached frequencies
offsets = Inputlist(1) #parse the input
current_frequency = 0  #start at 0
looping = True
counter = 0
while looping:
    counter+=1
    for offset in offsets:
        current_frequency += int(offset)
        if current_frequency not in frequency_list:
            frequency_list.add(current_frequency)
        else:
            looping = False
            break
print('Final result:',current_frequency)

Final result: 245


**Comments**:
- **first part** was fairly easy, a simple sum in the list worked really well, using list compehensions for added beauty
- **second part** was a bit confusing at first, since the code seem to be stuck in an infinite loop
- the problem was that I used list in the beginning, which had quite slow lookup times and the large number of iterations took too long. Changing to set made things work super quick!